In [ ]:
import pandas as pd
import json
from tqdm.notebook import tqdm
import random

In [ ]:
[{'id':df.iloc[i,0],'name':df.iloc[i,1],'latitude':df.iloc[i,2],'longtitude':df.iloc[i,3],'height':df.iloc[i,4]} for i in range(df.shape[0])]

In [ ]:
# 년월일 정렬
df=pd.read_csv('./폭행/폭행_사건별_리스트(주소추가).csv',index_col=0)
df.sort_values(by=['년','월','일','자치구','동'],inplace=True)
df.reset_index(inplace=True,drop=True)
data = {}
for year in df.년.unique():
    year_dict = {}
    for month in df[df.년==year].월.unique():
        month_dict = {}
        for day in df[(df.년==year)&(df.월==month)].일.unique():
            data_list = []
            for idx in df[(df.년==year)&(df.월==month)&(df.일==day)].index:
                data_dict = {
                    'gid': idx,
                    'gu': df.iloc[idx,3],
                    'dong': df.iloc[idx,4],
                    'addr': df.iloc[idx,5],
                    'lat': df.iloc[idx,6],
                    'lon': df.iloc[idx,7]
                }
                data_list.append(data_dict)
            month_dict[str(day)] = data_list
        year_dict[str(month)] = month_dict
    data[str(year)] = year_dict

with open('./jsons/년월일_정렬/폭행.json', 'w') as f:
    json.dump(data, f)

In [ ]:
# 년월구동 정렬
df=pd.read_csv('./폭행/폭행_사건별_리스트(주소추가).csv',index_col=0)
df.sort_values(by=['년','월','자치구','동','일'],inplace=True)
df.reset_index(inplace=True,drop=True)
data = {}
for year in tqdm(df.년.unique()):
    year_dict = {}
    for month in df[df.년==year].월.unique():
        month_dict = {}
        for gu in df[(df.년==year)&(df.월==month)].자치구.unique():
            gu_dict = {}
            for dong in df[(df.년==year)&(df.월==month)&(df.자치구==gu)].동.unique():
                data_list = []
                for idx in df[(df.년==year)&(df.월==month)&(df.자치구==gu)&(df.동==dong)].index:
                    data_dict = {
                        'gid':idx,
                        'day':int(df.iloc[idx,2]),
                        'addr':df.iloc[idx,5],
                        'lat':df.iloc[idx,6],
                        'lon':df.iloc[idx,7]
                    }
                    data_list.append(data_dict)
                gu_dict[str(dong)] = data_list
            month_dict[str(gu)] = gu_dict
        year_dict[str(month)] = month_dict
    data[str(year)] = year_dict

with open('./jsons/년월구동일_정렬/폭행.json', 'w') as f:
    json.dump(data, f)

In [ ]:
data = {}
for year in tqdm(df.년.unique()):
    year_dict = {}
    for month in df[df.년==year].월.unique():
        month_dict = {}
        for gu in df[(df.년==year)&(df.월==month)].자치구.unique():
            gu_dict = {}
            for dong in df[(df.년==year)&(df.월==month)&(df.자치구==gu)].동.unique():
                data_list = []
                for idx in df[(df.년==year)&(df.월==month)&(df.자치구==gu)&(df.동==dong)].index:
                    data_dict = {
                        'gid':idx,
                        'day':int(df.iloc[idx,2]),
                        'addr':df.iloc[idx,5],
                        'lat':df.iloc[idx,6],
                        'lon':df.iloc[idx,7]
                    }
                    data_list.append(data_dict)
                gu_dict[str(dong)] = data_list
            month_dict[str(gu)] = gu_dict
        year_dict[str(month)] = month_dict
    data[str(year)] = year_dict

with open('./jsons/년월구동일_정렬/폭행.json', 'w') as f:
    json.dump(data, f)

In [ ]:
a_json = open('./jsons/자치구별_월별_건수/폭행_자치구별_월별_건수.json')
a_dict = json.load(a_json)
for i in a_dict.keys():
    a_dict[i].append({'gid':a_dict[i][95]['gid']+1,'date':'predict','count':int(df.loc[i,'attack'])})
with open('./jsons/자치구별_월별_건수/폭행_자치구별_월별_건수.json', 'w') as f:
    json.dump(a_dict, f)

In [ ]:
df2=pd.read_csv('./성폭력/성폭력_월별_동별.csv',index_col=0)
df3=pd.read_csv('./동별_가중치.csv')
dongsu=df3.groupby(by='구').count().iloc[:,0].tolist()
ls=[]
for i in range(25):
    for _ in range(dongsu[i]):
        ls.append(df3.구.unique()[i])
df2=df2.T
df2.columns=ls

In [ ]:
# 동 max 값
crime=['강도','살인','성폭력','절도','폭행']
for k in crime:
    df2=pd.read_csv(f'./{k}/{k}_월별_동별.csv',index_col=0)
    df3=pd.read_csv('./동별_가중치.csv')
    dongsu=df3.groupby(by='구').count().iloc[:,0].tolist()
    ls=[]
    for i in range(25):
        for _ in range(dongsu[i]):
            ls.append(df3.구.unique()[i])
    df2=df2.T
    df2.columns=ls
    a_json = open(f'./jsons/사건_년월구동_정렬/{k}.json')
    a_dict = json.load(a_json)
    for i in a_dict.keys():
        for j in a_dict[i].keys():
            for l in a_dict[i][j].keys():
                a_dict[i][j][l]['max']=int(df2.loc[i+'.'+j,l].max())
    with open(f'./jsons/사건_년월구동_정렬/{k}2.json', 'w') as f:
        json.dump(a_dict, f)

In [ ]:
# 구 max 값
crime=['강도','살인','성폭력','절도','폭행']
for k in crime:
    df=pd.read_csv(f'./{k}/{k}_월별_자치구별.csv')
    a_json = open(f'./jsons/사건_년월구동_정렬/{k}2.json')
    a_dict = json.load(a_json)
    for i in a_dict.keys():
        for j in a_dict[i].keys():
            a_dict[i][j]['max']=int(df[i+'.'+j].max())
    with open(f'./jsons/사건_년월구동_정렬/{k}3.json', 'w') as f:
        json.dump(a_dict, f)

In [ ]:
df=pd.read_csv('./범죄발생시간.csv',index_col=0)
for k in range(5):
    globals()[f'df{k}']=df[df.columns[8*k:8*k+8]]
    globals()[f'df{k}'].columns.astype('object')
    globals()[f'df{k}'].columns=globals()[f'df{k}'].iloc[0,:]
    globals()[f'df{k}'].drop(index=globals()[f'df{k}'].index[0],inplace=True)
    globals()[f'df{k}'].columns=['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

    for j in range(8):
        probabilities=globals()[f'df{k}'][globals()[f'df{k}'].columns[j]].tolist()[1:-1]
        for i in range(len(probabilities)):
            probabilities[i]=probabilities[i]/sum(globals()[f'df{k}'][globals()[f'df{k}'].columns[j]].tolist()[1:-1])
        counts = {incident: 0 for incident in range(1, 9)}
        for i in range(globals()[f'df{k}'][globals()[f'df{k}'].columns[j]].tolist()[-1]):
            incident = random.choices(range(1, 9), probabilities)[0]
            counts[incident] += 1
        for i in range(1,9):
            globals()[f'df{k}'].iloc[i,j]=globals()[f'df{k}'].iloc[i,j] + counts[i]
    
    globals()[f'df{k}'].drop(globals()[f'df{k}'].index[-1],inplace=True)

    for j in range(globals()[f'df{k}'].shape[1]):
        for i in range(globals()[f'df{k}'].shape[0]-1):
            globals()[f'df{k}'].iloc[i+1,j]=globals()[f'df{k}'].iloc[i+1,j]/globals()[f'df{k}'].iloc[0,j]
    globals()[f'df{k}']=globals()[f'df{k}'].drop(globals()[f'df{k}'].index[0])

In [ ]:
crime=['절도','살인','강도','성폭력','폭행']
df=pd.read_csv('./절도/절도_사건별_리스트(주소추가).csv',index_col=0)
df['시간']=None
df=df[['년', '월', '일', '시간', '자치구', '동', '주소', '위도', '경도']]
df.sort_values(by=['년'],inplace=True)
for i in tqdm(range(df.shape[0])):
    p=df0[f'{df.iloc[i,0]}'].tolist()
    df.iloc[i,3]=f'{random.choices(range(0, 8), p)[0]*3+random.choice(range(0,3))}시{" 30분경" if random.choice(range(2))*30==30 else "경"}'
df.to_csv('./절도/절도_사건별_리스트(시간추가).csv')

In [ ]:
crime=['절도','살인','강도','성폭력','폭행']
for crm in crime:
    a_json = open(f'./jsons/사건_년월구동_정렬/{crm}.json')
    a_dict = json.load(a_json)
    for a in tqdm(a_dict.keys()):
        p=globals()[f'df{crime.index(crm)}'][a].tolist()
        for b in a_dict[a].keys():
            for c in list(a_dict[a][b].keys())[:-1]:
                for d in list(a_dict[a][b][c].keys())[:-1]:
                    for e in range(len(a_dict[a][b][c][d])):
                        a_dict[a][b][c][d][e]['time']=f'{random.choices(range(0, 8), p)[0]*3+random.choice(range(0,3))}시{" 30분경" if random.choice(range(2))*30==30 else "경"}'
    with open(f'./jsons/사건_년월구동_정렬/{crm}2.json', 'w') as f:
        json.dump(a_dict, f)

In [ ]:
crime=['attack','murder','robber','sexual','theft']
dongwgt=pd.read_csv('./동별_가중치.csv')
dongwgt.dropna(axis=0,inplace=True)
for crm in crime:
    globals()[crm]=pd.DataFrame(index=dongwgt[['구','동']],columns=[f'2022.{i}' for i in range(1,13)])
dongwgt.set_index(['구'],inplace=True)
dongwgt=dongwgt.T
for crm in tqdm(crime):
    df=pd.read_csv(f'./예측/pred_{crm}.csv',index_col=1)
    df.drop(columns=df.columns[0],inplace=True)
    df=df.astype('int')
    df.columns=[f'2022.{i}' for i in range(1,13)]
    for i in dongwgt.columns.unique():
        p=dongwgt.loc['가중치',i].tolist()
        for j in range(1,13):
            counts = {dong: 0 for dong in dongwgt.loc['동',i].tolist()}
            for _ in range(df.loc[i,f'2022.{j}']):
                dong = random.choices(dongwgt.loc['동',i].tolist(), p)[0]
                counts[dong] += 1
            for dong in dongwgt.loc['동',i].tolist():
                globals()[crm].loc[[(i,dong)],[f'2022.{j}']]=counts[dong]

In [ ]:
crime=['attack','murder','robber','sexual','theft']
for crm in crime:
    globals()[f'cases_{crm}']=pd.DataFrame(index=range(globals()[crm].sum().sum()),columns=range(3))
    globals()[f'cases_{crm}'].columns=['월','구','동']
    n=0
    for i in range(globals()[crm].shape[0]):
        for j in range(globals()[crm].shape[1]):
            for _ in range(globals()[crm].iloc[i,j]):
                globals()[f'cases_{crm}'].iloc[n,0]=globals()[crm].columns[j].split('.')[1]
                globals()[f'cases_{crm}'].iloc[n,1]=globals()[crm].index[i][0]
                globals()[f'cases_{crm}'].iloc[n,2]=globals()[crm].index[i][1]
                n+=1

In [ ]:
a_json = open(f'./jsons/사건_년월구동_정렬/절도.json')
a_dict = json.load(a_json)
i=a_dict['2021']['12'][list(a_dict['2021']['12'].keys())[-2]][list(a_dict['2021']['12'][list(a_dict['2021']['12'].keys())[-2]].keys())[-2]][-1]['gid']
cases_theft['월']=cases_theft['월'].astype('int32')
cases_theft.sort_values(by=['월','구','동'],inplace=True)
cases_theft.index=range(i+1,i+1+cases_theft.shape[0])
cases_theft['월']=cases_theft['월'].astype('object')
data={}
for month in cases_theft.월.unique():
    month_dict={}
    for gu in cases_theft[cases_theft.월==month].구.unique():
        gu_dict={}
        for dong in cases_theft[(cases_theft.월==month)&(cases_theft.구==gu)].동.unique():
            data_list=[]
            for idx in cases_theft[(cases_theft.월==month)&(cases_theft.구==gu)&(cases_theft.동==dong)].index:
                data_dict={'gid':idx}
                data_list.append(data_dict)
            gu_dict[str(dong)]=data_list
        gu_dict['max']=int(cases_theft[(cases_theft.월==month)&(cases_theft.구==gu)].groupby(by='동').count().max()[0])
        month_dict[str(gu)]=gu_dict
    month_dict['max']=int(cases_theft[cases_theft.월==month].groupby(by='구').count().max()[0])
    data[str(month)]=month_dict
a_dict['2022']=data
with open(f'./jsons/사건_년월구동_정렬/절도2.json', 'w') as f:
    json.dump(a_dict, f)

In [ ]:
# 서울시_월별_건수
a_json = open(f'./jsons/서울시_월별_건수/폭행_서울시_월별_건수.json')
a_dict = json.load(a_json)
idx=a_dict[-1]['gid']+1
for i in range(attack.shape[1]):
    a_dict.append({'gid':int(idx+i),'date':attack.columns[i],'count':attack[attack.columns[i]].sum()})
with open(f'./jsons/서울시_월별_건수/폭행_서울시_월별_건수.json', 'w') as f:
    json.dump(a_dict, f)

In [ ]:
a_json = open(f'./jsons/동별_월별_건수/절도_동별_월별_건수.json')
a_dict = json.load(a_json)
for dong in a_dict.keys():
    for i in range(theft.shape[1]):
        a_dict[dong].append({'gid':int(96+i),'date':theft.columns[i],'count':int(theft.loc[[(dong.replace("'",'').split())],theft.columns[i]])})
i=0
for dong in a_dict.keys():
    for j in range(a_dict[dong].__len__()):
        a_dict[dong][j]['gid']=i
        i+=1
with open(f'./jsons/동별_월별_건수/절도_동별_월별_건수.json', 'w') as f:
    json.dump(a_dict, f)

In [ ]:
attack['구']=[index[0] for index in attack.index]
attack2=attack.groupby(by='구').sum()
a_json = open(f'./jsons/자치구별_월별_건수/폭행_자치구별_월별_건수.json')
a_dict = json.load(a_json)
for gu in a_dict.keys():
    for i in range(attack2.shape[1]):
        a_dict[gu].append({'gid':0,'date':attack2.columns[i],'count':int(attack2.loc[gu,attack2.columns[i]])})
i=0
for gu in a_dict.keys():
    for j in range(a_dict[gu].__len__()):
        a_dict[gu][j]['gid']=i
        i+=1
with open(f'./jsons/자치구별_월별_건수/폭행_자치구별_월별_건수.json', 'w') as f:
    json.dump(a_dict, f)